Я вижу два решения:

Для простоты я бы сделал 2 класса датасета
Если нужно кодировать лишь последовательность букв, он и хранит последовательности букв сразу и не хранит коордианты

В обоих случаях декодер оперирует эмбеддингами букв текста

<h3>1. На вход энкодера x, y, t, dx/dt, dy/st, x'', y'', keybard_key_embedding</h3>

**Что делать, если ближайшая клавиша неалфавитная (пунктуация, клавиши-действия)?**

Добавлю для всех неалфавитных клавиш один специальный токен. Хотя, возможно,
лучше добавить отдельный токен для каждой клавиши. Кажется, в кавиатуре схожесть
клавиш определяется тем, насколько часто они встречаются рядом друг с другом.
Тогда может быть важно отличать enter, который близок к бкве `э`, например,
от `caps lock`, который близок к `ф`. Отмечу, что схожесть в данном случае - это
не просто физическое расстояние между клавишами (хотя отчасти и так), но скорее, похожесть
клавиш `a` и `b` означает, что для последовательностей вида `letter1`, `letter2` ... `letterX`, `letterN`
если вероятность, что `letterX` = `a` велика, то вероятность, что `letterX` = `b` тоже велика.


**Где происходит инициализация токенизатора?**
я бы вынес токенезатор вне датасета и передавал бы его в конструктор датасета.


для каждой раскладки свои instance'ы датасета и модели.



<h3> 2. На вход энкодера последовательность клавиш клавиатуры </h3>

Если ближайшая клавиша неалфавитная **пропускать**

**Где происходит инициализация токенизатора?**


один instance датасета и одна модель для всех раскладок.


Реализовывать ли для каждого варианта отдельный токенизатор:

У нас может быть различное количество токенов: в некотоорых раскладках отсутствует символ "ъ", например

Когда датасет содержит лишь одну раскладку, токенизатор должен учесть символы из одной раскладки. Когда датасет содержит несколько раскладок, токенизатор должен учесть символы из всех раскладок.

Кажется, что варьируется только наличие 'ъ' и 'ё'. Во-первых, не ясно нужны ли эти символы. Есть желание заменять 'ё' на 'е', а 'ъ' на 'ь'. 

Кажется, нас совершенно устраивает токенизатор, содержащий все буквы русского языка, включая 'ё' и 'ъ'. Наличие пары лишних токенов незначительно увеличит размер эмбеддинг-матрицы, но не повлияет на обучение модели.

In [104]:
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, IterableDataset

# from model import SwipeCurveEncoderTransformer

In [108]:
data_root = "../data/data_separated_grid"

In [109]:
from typing import List, Optional

class CharLevelTokenizerv1:
    """
    Tokenizes a word into a list of integers.

    Toknized word is padded to the max_word_len.

    Guarantees that <sos> and <pad> are tokens with `vocab_len - 1`
    and `vocab_len - 2` indices respectively.
    This is useful because model never generates <sos> and <pad> tokens.
    """
    def __init__(self, vocab_path):
        self.char_to_idx = {}
        self.idx_to_char = {}
        self.max_word_len = None  # is set in _build_vocab
        self._build_vocab(vocab_path)

    def _build_vocab(self, vocab_path):
        self.max_word_len = 0
        special_tokens = ["<eos>", "<pad>", "<unk>", "<sos>"]
        unique_chars = set()

        with open(vocab_path, "r", encoding="utf-8") as f:
            vocab = f.read().split("\n")
            for word in vocab:
                self.max_word_len = max(self.max_word_len, len(word) + 2)  # + <sos> and <eos>
                for char in word:
                    unique_chars.add(char)
                    
        unique_chars_list = sorted(list(unique_chars)) + special_tokens
        self.char_to_idx = {char: idx for idx, char in enumerate(unique_chars_list)}
        self.idx_to_char = {idx: char for idx, char in enumerate(unique_chars_list)}

    def _tokenize_word(self, word):
        """
        Tokenizes a word into a list of integers.
        """
        tokenized_word = []
        tokenized_word.append(self.char_to_idx["<sos>"])
        for char in word:
            default: int = self.char_to_idx['<unk>']
            tokenized_word.append(self.char_to_idx.get(char, default))
        tokenized_word.append(self.char_to_idx["<eos>"])
        return tokenized_word
    
    def _pad(self, tokenized_word):
        """
        Pads a word to the max_word_len.
        """
        return tokenized_word + [self.char_to_idx["<pad>"]] * (self.max_word_len - len(tokenized_word))
    
    def tokenize(self, word):
        """
        Tokenizes a word and pads it to the max_word_len.
        """
        token_seq = torch.tensor(self._pad(self._tokenize_word(word)))
        mask = torch.ones(self.max_word_len, dtype=torch.bool)
        mask[:len(word)+2] = False
        return token_seq, mask
    
    def decode(self, token_seq):
        """
        Decodes a tokenized word into a string.
        """
        return "".join([self.idx_to_char[int(idx)] for idx in token_seq])

In [110]:
import pickle

word_tokenizer = CharLevelTokenizerv1(os.path.join(data_root, "voc.txt"))

word_tokenizer_save_path = "../data/data_separated_grid/word_tokenizer.pkl"

with open(word_tokenizer_save_path, 'wb') as f:
    pickle.dump(word_tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(word_tokenizer_save_path, 'rb') as f:
    word_tokenizer = pickle.load(f)

In [111]:
print(word_tokenizer.idx_to_char)

{0: '-', 1: 'а', 2: 'б', 3: 'в', 4: 'г', 5: 'д', 6: 'е', 7: 'ж', 8: 'з', 9: 'и', 10: 'й', 11: 'к', 12: 'л', 13: 'м', 14: 'н', 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф', 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 27: 'ъ', 28: 'ы', 29: 'ь', 30: 'э', 31: 'ю', 32: 'я', 33: '<eos>', 34: '<pad>', 35: '<unk>', 36: '<sos>'}


In [112]:
class KeyboardTokenizerv1:
    
    i2t = ['а', 'б', 'в', 'г', 'д', 'е', 'ë', 'ж', 'з', 'и', 'й',
           'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф',
           'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
           '-', '<unk>', '<pad>']
    
    t2i = {t: i for i, t in enumerate(i2t)}

    def get_token(self, char):
        return self.t2i.get(char, self.t2i['<unk>'])

In [125]:
import json
from typing import Optional, List, Tuple, Dict
import array
import numpy as np

import torch
from torch.utils.data import Dataset
from tqdm import tqdm


class NeuroSwipeDatasetv1(Dataset):
    """
    Dataset class for NeuroSwipe dataset.
    The dataset file weights over 3 GB and contains over 6 million swipe gestures.
    """

    def __init__(self,
                 data_path: str,
                 grid: dict,
                 kb_tokenizer: KeyboardTokenizerv1,
                 max_traj_len: int,
                 word_tokenizer: CharLevelTokenizerv1,  # should contain max word len
                 include_time: bool = True,
                 include_velocities: bool = True,
                 include_accelerations: bool = True,
                 total: Optional[int] = None):
        """
        Argsuments:
        -----------
        data_path (string): Path to the NeuroSwipe dataset in JSON format.
            A custom version of the dataset is used:
            "grid" property is replaced with "grid_name" property.
        """
        if include_accelerations and not include_velocities:

            raise ValueError("Accelerations are supposed \
                             to be an addition to velocities. Add velocities.")

        self.max_traj_len = max_traj_len
        self.include_velocities = include_velocities
        self.include_accelerations = include_accelerations
        self.include_time = include_time

        self.word_tokenizer = word_tokenizer

        self.kb_width = grid['width']
        self.kb_height = grid['height']

        self._coord_to_kb_label = self._get_coord_to_kb_label(grid)

        self.data_list = []
        self._set_data(data_path, grid, kb_tokenizer, self.data_list, total = total)
    

    def _get_key_center(self, hitbox: Dict[str, int]) -> Tuple[int, int]:
        x = hitbox['x'] + hitbox['w'] / 2
        y = hitbox['y'] + hitbox['h'] / 2
        return x, y

    def _get_kb_label_without_map(self, x, y, grid: dict) -> str:
        nearest_kb_label = None
        min_dist = float("inf")
        for key in grid['keys']:
            key_x, key_y = self._get_key_center(key['hitbox'])
            dist = (x - key_x)**2 + (y - key_y)**2
            if dist < min_dist:
                min_dist = dist
                if 'label' in key:
                    nearest_kb_label = key['label']
                elif 'action' in key:
                    nearest_kb_label = key['action']  # tokenizer will covert it to <unk>
                else:
                    raise ValueError("Key has no label or action")
        return nearest_kb_label


    def _get_coord_to_kb_label(self, grid: dict) -> np.array:
        coord_to_kb_label = np.chararray((grid['width'], grid['height']), unicode=True) # 1080 x 640 in our case
        coord_to_kb_label.fill('')

        for key in grid['keys']:
            x_left = key['hitbox']['x']
            x_right = x_left + key['hitbox']['w']
            y_top = key['hitbox']['y']
            y_bottom = y_top + key['hitbox']['h']

            # tokenizer will covert actions to <unk>
            label = key['label'] if 'label' in key else key['action']

            if len(label) > 1:
                print(f"Warning: label '{label}' is substituted with {label[0]}")

            coord_to_kb_label[x_left:x_right, y_top:y_bottom] = label


        for x in range(grid['width']):
            for y in range(grid['height']):
                if coord_to_kb_label[x, y] != '':
                    continue
                        
                coord_to_kb_label[x, y] = self._get_kb_label_without_map(x, y, grid)

        return coord_to_kb_label
    
    def coord_to_kb_label(self, x, y, grid: dict) -> str:
        if x < 0 or x >= self.kb_width or y < 0 or y >= self.kb_height:
            return self._get_kb_label_without_map(x, y, grid)
        else:
            return self._coord_to_kb_label[x, y]
            

    def _set_data(self,
                  data_path: str,
                  kb_keys: str,
                  kb_tokenizer,
                  data_list: list,
                  total: Optional[int] = None):
        with open(data_path, "r", encoding="utf-8") as json_file:
            for line in tqdm(json_file, total = total):
                data_list.append(self._get_data_from_json_line(line, kb_keys, kb_tokenizer))


    def _get_dx_dt(self,
                   X: torch.tensor,
                   T: torch.tensor,
                   len: int) -> List[float]:
        """
        Calculates dx/dt for a list of x coordinates and a list of t coordinates.

        Arguments:
        ----------
        X : torch.tensor
            x (position) coordinates.
        T : torch.tensor
            T[i] = time from the beginning of the swipe corresponding to X[i].
        len : int
            Length of the swipe trajectory. Indexes greater than len are ignored.

        """
        dx_dt = torch.zeros_like(X)
        # dx_dt[1:-1] = (X[2:] - X[:-2]) / (T[2:] - T[:-2])
        dx_dt[1:len-1] = (X[2:len] - X[:len-2]) / (T[2:len] - T[:len-2])

        # Example:
        # x0 x1 x2 x3
        # t0 t1 t2 t3
        # dx_dt[0] = 0
        # dx_dt[1] = (x2 - x0) / (t2 - t0)
        # dx_dt[2] = (x3 - x1) / (t3 - t1)
        # dx_dt[3] = 0


        # if True in torch.isnan(dx_dt):
        #     print(dx_dt)
        #     raise ValueError("dx_dt contains NaNs")

        return dx_dt
    
    def _get_data_from_json_line(self, line, grid, kb_tokenizer) -> Tuple[list, list, list, str]:
        """
        Parses a JSON line and returns a dictionary with data.
        """
        data = json.loads(line)
        word: str = data['word']

        X = array.array('h', data['curve']['x'])
        Y = array.array('h', data['curve']['y'])
        T = array.array('h', data['curve']['t'])        

        kb_labels = [self.coord_to_kb_label(x, y, grid) for x, y in zip(X, Y)]
        kb_tokens = [kb_tokenizer.get_token(label) for label in kb_labels]
        kb_tokens += [kb_tokenizer.get_token('<pad>')] * (self.max_traj_len - len(kb_labels))
        kb_tokens = array.array('h', kb_tokens)

        return X, Y, T, word, kb_tokens

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        X_list, Y_list, T_list, word, kb_tokens = self.data_list[idx]

        X = torch.zeros(self.max_traj_len, dtype=torch.float32)
        Y = torch.zeros(self.max_traj_len, dtype=torch.float32)
        T = torch.zeros(self.max_traj_len, dtype=torch.float32)
        
        X[:len(X_list)] = torch.tensor(X_list, dtype=torch.float32) / self.kb_width
        Y[:len(Y_list)] = torch.tensor(Y_list, dtype=torch.float32) / self.kb_height
        T[:len(T_list)] = torch.tensor(T_list, dtype=torch.float32)

        xyt = torch.cat(
            (
                X.reshape(-1, 1),
                Y.reshape(-1, 1),
            ),
            axis = 1
        )

        if self.include_time:
            xyt = torch.cat(
                (
                    xyt,
                    T.reshape(-1, 1)
                ),
                axis = 1
            )

        traj_len = len(X_list)

        if self.include_velocities:
            dx_dt = self._get_dx_dt(X, T, traj_len)
            dy_dt = self._get_dx_dt(Y, T, traj_len)
            xyt = torch.cat(
                [
                    xyt,
                    dx_dt.reshape(-1, 1),
                    dy_dt.reshape(-1, 1)
                ],
                axis = 1
            )

        if self.include_accelerations:
            d2x_dt2 = self._get_dx_dt(dx_dt, T, traj_len)
            d2y_dt2 = self._get_dx_dt(dy_dt, T, traj_len)
            xyt = torch.cat(
                [
                    xyt,
                    d2x_dt2.reshape(-1, 1),
                    d2y_dt2.reshape(-1, 1)
                ],
                axis = 1
            )

        traj_pad_mask = torch.ones(self.max_traj_len, dtype=torch.bool)
        traj_pad_mask[:len(X_list)] = False

        char_seq, word_mask = self.word_tokenizer.tokenize(word)
        
        word_mask = word_mask[:-1]

        decoder_in_char_seq = char_seq[:-1]
        decoder_out_char_seq = char_seq[1:]

        kb_tokens = torch.tensor(kb_tokens, dtype=torch.int64)
    
        return xyt, kb_tokens, traj_pad_mask, decoder_in_char_seq, word_mask, decoder_out_char_seq

In [126]:
def get_grid(grid_name: str, grids_path: str) -> dict:
    with open(grids_path, "r", encoding="utf-8") as f:
        return json.load(f)[grid_name]

In [127]:
sample_data = os.path.join(data_root, "sample_deleteme__default_only.jsonl")
grid_path =  os.path.join(data_root, "gridname_to_grid.json")
grid_name = "default"

grid = get_grid(grid_name, grid_path)
kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(data_root, "voc.txt"))


dataset = NeuroSwipeDatasetv1(
    data_path = sample_data,
    grid = grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    total = 1000
)

100%|██████████| 1000/1000 [00:00<00:00, 3758.91it/s]


In [116]:
i = 40
xyt, kb_tokens, traj_pad_mask, dec_in_char_seq, word_pad_mask, dec_out_char_seq = dataset[i]
print(xyt.shape, kb_tokens.shape, traj_pad_mask.shape, dec_in_char_seq.shape, word_pad_mask.shape, dec_out_char_seq.shape)

torch.Size([299, 6]) torch.Size([299]) torch.Size([299]) torch.Size([35]) torch.Size([35]) torch.Size([35])


In [124]:
np.set_printoptions(threshold=1000000)
print(dataset._coord_to_kb_label)

[['й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 

In [59]:
from typing import Callable
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

class SwipeCurveTransformerEncoderv1(nn.Module):
    """
    Transformer-based Curve encoder takes in a sequence of vectors and creates a representation
    of a swipe gesture on a samrtphone keyboard.
    Each vector contains information about finger trajectory at a time step.
    It contains:
    * x coordinate
    * y coordinate
    * Optionally: t
    * Optionally: dx/dt
    * Optionally: dy/dt
    * Optionally: keyboard key that has x and y coordinates within its boundaries
    """

    def __init__(self, input_size, d_model,
                 dim_feedforward, num_layers, num_heads_first, num_heads_other,
                 dropout = 0.1):
        """
        Arguments:
        ----------
        input_size: int
            Size of input vectors.
        d_model: int
            Size of the embeddings (output vectors).
            Should be equal to char embedding size of the decoder.
        dim_feedforward: int
        num_layers: int
            Number of encoder layers including the first layer.

        """
        super().__init__()
        # self.pos_encoder = PositionalEncoding(input_size, dropout)
        self.first_encoder_layer = nn.TransformerEncoderLayer(
            input_size, num_heads_first, dim_feedforward, dropout)
        self.liner = nn.Linear(input_size, d_model)  # to convert embedding to d_model size
        num_layer_after_first = num_layers - 1
        if num_layer_after_first > 0:
            encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads_other, dim_feedforward, dropout)
            self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        else:
            self.transformer_encoder = None
    

    def forward(self, x, pad_mask: torch.tensor):
        # x = self.pos_encoder(x)
        x = self.first_encoder_layer(x, src_key_padding_mask=pad_mask)
        x = self.liner(x)
        if self.transformer_encoder:
            x = self.transformer_encoder(x, src_key_padding_mask=pad_mask)
        return x



class SwipeCurveTransformerDecoderv1(nn.Module):
    """
    Decodes a swipe gesture representation into a sequence of characters.

    Uses decoder transformer with masked attention to prevent the model from cheating.
    """

    def __init__(self, char_emb_size, n_classes, nhead, num_decoder_layers,
                 dim_feedforward, dropout, activation = F.relu):
        super().__init__()

        self.decoder_layer = nn.TransformerDecoderLayer(
            char_emb_size, nhead, dim_feedforward, dropout, activation)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_decoder_layers)
        self.out = nn.Linear(char_emb_size, n_classes)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, memory, tgt_mask, memory_key_padding_mask, tgt_key_padding_mask):
        x = self.transformer_decoder(x,
                                     memory,
                                     tgt_mask=tgt_mask,
                                     memory_key_padding_mask=memory_key_padding_mask,
                                     tgt_key_padding_mask=tgt_key_padding_mask)
        x = self.out(x)
        # x = self.softmax(x)
        return x


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int, dropout: float = 0.0):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class SwipeCurveTransformer(nn.Module):
    """
    Seq2seq model. Encodes a sequence of points of a
    swipe-keyboard-typing gesture into a sequence of characters.

    n_output_classes = char_vocab_size - 2 because <pad> and <sos>
    tokens are never predicted.
    """

    def _get_mask(self, max_seq_len: int):
        """
        Returns a mask for the decoder transformer.
        """
        mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

    def __init__(self,
                 n_coord_feats: int,
                 char_emb_size: int,
                 char_vocab_size: int,
                 key_emb_size: int,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 dim_feedforward: int,
                 num_heads: int,
                 dropout:float,
                 char_embedding_dropout: float,
                 key_embedding_dropout: float,
                 max_out_seq_len: int,
                 max_curves_seq_len: int,
                 activation: Callable = F.relu,
                 device: str = None):
        super().__init__()

        input_size = n_coord_feats + key_emb_size

        curv_emb_size = char_emb_size

        self.char_embedding_dropout = nn.Dropout(char_embedding_dropout)
        self.key_embedding_dropout = nn.Dropout(key_embedding_dropout)
        
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_size)
        self.key_embedding = nn.Embedding(char_vocab_size, key_emb_size)

        self.encoder = SwipeCurveTransformerEncoderv1(
            input_size, curv_emb_size, dim_feedforward,
            num_encoder_layers, num_heads, dropout)
        
        self.char_pos_encoder = PositionalEncoding(
            char_emb_size, max_out_seq_len)
        
        self.key_pos_encoder = PositionalEncoding(
            key_emb_size, max_curves_seq_len)
        
        n_classes = char_vocab_size - 2  # <sos> and <pad> are not predicted
        self.decoder = SwipeCurveTransformerDecoderv1(
            char_emb_size, n_classes, num_heads,
            num_decoder_layers, dim_feedforward, dropout, activation)

        self.mask = self._get_mask(max_out_seq_len)

    def forward(self, x, kb_tokens, y, x_pad_mask, y_pad_mask):
        kb_k_emb = self.key_embedding(kb_tokens)  # keyboard key
        kb_k_emb = self.key_embedding_dropout(kb_k_emb)
        kb_k_emb = self.key_pos_encoder(kb_k_emb)
        x = torch.cat((x, kb_k_emb), axis = -1)
        x = self.encoder(x, x_pad_mask)
        y = self.char_embedding(y)
        y = self.char_embedding_dropout(y)
        y = self.char_pos_encoder(y)
        y = self.decoder(y, x, self.mask, x_pad_mask, y_pad_mask)
        return y

In [45]:
# def get_mask(max_seq_len: int):
#     """
#     Returns a mask for the decoder transformer.
#     """
#     mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1)
#     mask = mask.masked_fill(mask == 1, float('-inf'))
#     return mask

# mask = get_mask(5)
# print(mask)

# >>>
# tensor([[0., -inf, -inf, -inf, -inf],
#         [0., 0., -inf, -inf, -inf],
#         [0., 0., 0., -inf, -inf],
#         [0., 0., 0., 0., -inf],
#         [0., 0., 0., 0., 0.]])

In [46]:
seq_len = 32
batch_size = 10
in_features = 40



encoder = SwipeCurveTransformerEncoderv1(
    input_size=in_features,
    d_model=128,
    dim_feedforward=128,
    num_layers=1,
    num_heads_first=2,
    num_heads_other=4,
    dropout=0.1)



pad_mask = torch.zeros(batch_size, seq_len, dtype=torch.bool)
# as if each batch contains 22 actual sequnce elements and 10 padding elements
pad_mask[:, 10:] = True


encoded = encoder(torch.rand(seq_len, batch_size, in_features), pad_mask)
torch.set_printoptions(threshold=100_000)

encoded.transpose_(0,1)

if True in torch.isnan(encoded):
    print(encoded)
    raise ValueError("encoded contains NaNs")

In [47]:
curves_seq_len = 20
chars_seq_len = 14
batch_size = 10
char_emb_size = 32
n_classes = 5

decoder = SwipeCurveTransformerDecoderv1(
    char_emb_size=char_emb_size,
    n_classes=n_classes,
    nhead=2,
    num_decoder_layers=2,
    dim_feedforward=128,
    dropout=0.1)

def get_mask(max_seq_len: int):
    """
    Returns a mask for the decoder transformer.
    """
    mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1)
    mask = mask.masked_fill(mask == 1, float('-inf'))
    return mask

x = torch.rand(chars_seq_len, batch_size, char_emb_size)
memory = torch.rand(curves_seq_len, batch_size, char_emb_size)
tgt_mask = get_mask(chars_seq_len)
memory_key_padding_mask = torch.zeros(batch_size, curves_seq_len, dtype=torch.bool)
memory_key_padding_mask[:, 15:] = True
tgt_key_padding_mask = torch.zeros(batch_size, chars_seq_len, dtype=torch.bool)
tgt_key_padding_mask[:, 10:] = True


decoded = decoder(
    x,
    memory,
    tgt_mask=tgt_mask,
    memory_key_padding_mask=memory_key_padding_mask,
    tgt_key_padding_mask=tgt_key_padding_mask)

if True in torch.isnan(decoded):
    print(decoded)
    # raise ValueError("decoded contains NaNs")

print(decoded.shape)

torch.Size([14, 10, 5])


c:\Users\proshian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [48]:
word_char_tokenizer = CharLevelTokenizerv1("../data/data_separated_grid/voc.txt")

In [49]:
word_char_tokenizer.tokenize("троллейбус")

(tensor([36, 19, 17, 15, 12, 12,  6, 10,  2, 20, 18, 33, 34, 34, 34, 34, 34, 34,
         34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34]),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True]))

In [50]:
print(word_char_tokenizer.char_to_idx.keys())

dict_keys(['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', '<eos>', '<pad>', '<unk>', '<sos>'])


In [51]:
len(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

36

In [52]:
len(word_char_tokenizer.tokenize('информационно')[0])

36

In [53]:
word_char_tokenizer.decode(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

'<sos>информационно-телекоммуникационной<eos>'

In [54]:
word_char_tokenizer.max_word_len - 1

35

In [60]:
max_out_seq_len = word_char_tokenizer.max_word_len - 1

transformer = SwipeCurveTransformer(
    n_coord_feats=6,
    char_emb_size=128,
    char_vocab_size=len(word_char_tokenizer.char_to_idx),
    key_emb_size=32,
    num_encoder_layers=1,
    num_decoder_layers=2,
    dim_feedforward=128,
    num_heads=1,
    dropout=0.1,
    max_out_seq_len=max_out_seq_len,
    max_curves_seq_len=299)

In [61]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, shuffle=True)

for xyt, kb_tokens, traj_pad_mask, dec_in_char_seq, word_pad_mask, dec_out_char_seq in loader:
    xyt.transpose_(0, 1)  # (curves_seq_len, batch_size, n_coord_feats)
    kb_tokens.transpose_(0, 1) # (curves_seq_len, batch_size)
    dec_in_char_seq.transpose_(0, 1)  # (chars_seq_len - 1, batch_size)
    dec_out_char_seq.transpose_(0, 1)  # (chars_seq_len - 1, batch_size)

    # traj_pad_mask.shape: (batch_size, curves_seq_len)
    # word_pad_mask.shape: (batch_size, chars_seq_len - 1)

    char_seq_pred = transformer(
        xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask)
    break

c:\Users\proshian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [62]:
char_seq_pred.transpose(0,1)[0].shape

if True in torch.isnan(char_seq_pred):
    print(char_seq_pred)

In [63]:
print(char_seq_pred.shape)  # max_word_len - 1, batch_size, char_vocab_size - 2 (omitting <sos> and <pad>)

torch.Size([35, 10, 35])
